## Roboticia-quattro : Tutorial 1 - the leg primitive

### How to move the legs using Pypot Primitives

Instanciate the robot :

In [ ]:
from roboticia_quattro import RoboticiaQuattro

If you want to open your robot in the Vrep simulator (http://www.coppeliarobotics.com/) :

In [2]:
robot = RoboticiaQuattro(simulator='vrep')

IOError: No such file or directory: 'roboticia_quattro.ttt'

If you are lucky enough to have a real robot (http://www.roboticia.com) :

In [ ]:
robot = RoboticiaQuattro()

Your robot have motors and sensors :

In [121]:
robot.motors

[<DxlMotor name=m11 id=11 pos=0.0>,
 <DxlMotor name=m12 id=12 pos=-20.0>,
 <DxlMotor name=m13 id=13 pos=20.0>,
 <DxlMotor name=m21 id=21 pos=-0.0>,
 <DxlMotor name=m22 id=22 pos=-20.0>,
 <DxlMotor name=m23 id=23 pos=20.0>,
 <DxlMotor name=m31 id=31 pos=0.0>,
 <DxlMotor name=m32 id=32 pos=20.0>,
 <DxlMotor name=m33 id=33 pos=-20.0>,
 <DxlMotor name=m41 id=41 pos=0.0>,
 <DxlMotor name=m42 id=42 pos=20.0>,
 <DxlMotor name=m43 id=43 pos=-20.0>]

In [126]:
robot.sensors

[<ForceSensor name=f1 force = 40.6818628311>,
 <ForceSensor name=f2 force = 15.3728663921>,
 <ForceSensor name=f3 force = 20.0486108661>,
 <ForceSensor name=f4 force = 37.7162396908>]

Obviously you can access the motors and change angles, but if you do so without any control, pretty sure your robot will fall !

In [133]:
robot.m12.goto_position(-80,2)

In a vrep simulated environnement you can restart the simulation :

In [134]:
robot.reset_simulation()

It is why, you need a model to control the legs. For the inverse kinematic model of the leg, we are using the cosinus law. Each leg is like a triangle, and thanks to the cosinus law you can resolve the triangle : https://en.wikipedia.org/wiki/Law_of_cosines

In [135]:
import math as mt
import time

In [136]:
def AK_side(side_a,side_b,side_c):
    """
    Take the 3 side of a triangle and return the angles
    """
    alpha = mt.acos((side_b**2 + side_c**2 - side_a**2)/(2*side_b*side_c))
    beta = mt.acos((side_a**2 + side_c**2 - side_b**2)/(2*side_a*side_c))
    gamma = mt.pi - alpha - beta
    return (alpha,beta,gamma)

def AK_angle(side_a,side_b,gamma):
    """
    Take 2 sides and 1 angle of a triangle and return the missing angles and sides
    """
    side_c = mt.sqrt (side_a**2 + side_b**2 - 2*side_a*side_b*mt.cos(gamma))
    alpha = mt.acos((side_b**2 + side_c**2 - side_a**2)/(2*side_b*side_c))
    beta= mt.pi - alpha - gamma
    return (side_c,alpha,beta)

The Leg primitive will now make the link between 4 attributes of the leg (h, d, b, f) and the right angles for the 3 motors. The loop is updated at a given frequency. In our example, we use an update frequncy of 50 Hz.

In [137]:
from pypot.primitive import LoopPrimitive

In [138]:
class Leg(LoopPrimitive):
    def __init__(self,robot,leg,refresh_freq=50):
        self.robot = robot
        LoopPrimitive.__init__(self, robot,refresh_freq)
        # you should never access directly to the motors in a primitive, because this is the goal of the Primitives manager 
        # to manage this acces for all the primitives : https://poppy-project.github.io/pypot/primitive.html
        fake_motors = getattr(self.robot, leg)
        self.knee = fake_motors[2]
        self.hip = fake_motors[1]
        self.hip_lateral = fake_motors[0]
        self.motors = fake_motors
        
        # size of segment's leg            
        self.shin = 63
        self.thigh = 55
        self.pelvis = 38
    
    @property      
    def get_pos(self):
        (side_c,alpha,beta) = AK_angle(self.thigh,self.shin,mt.pi-abs(mt.radians(self.knee.present_position)))
        # what knee flexion
        flex = mt.copysign(1,self.knee.present_position)
        # calcul de l'angle beta_2 entre side_c et la veticale
        beta_2 = mt.radians(self.hip.present_position)+beta*flex
        theta = mt.radians(self.hip_lateral.present_position)
        
        high_leg = mt.cos(beta_2)*side_c+self.pelvis
        
        high = mt.cos(theta)*high_leg
        distance = mt.sin(beta_2)*side_c
        balance = mt.sin(theta)*high_leg
        
        return (high,distance,flex,balance)
      
    
    def set_pos(self,h,d,b):
        high_leg = mt.sqrt(h**2+b**2)
        side_c = mt.sqrt((high_leg-self.pelvis)**2+d**2)
        beta_2 = mt.asin(d/side_c)
        (alpha,beta,gamma) = AK_side(self.thigh,self.shin,side_c)
        
        angle_hip = mt.degrees(beta_2 - beta*self.f)
        angle_knee = self.f*mt.degrees(mt.pi - gamma)
        angle_hip_lateral = mt.degrees(mt.asin(b/high_leg))
            
        return(angle_hip_lateral, angle_hip, angle_knee)
        
    def h_limit(self,d):
        pass
        
    def d_limit(self,h):
        pass
        
    def setup(self):
        (self.h,self.d,self.f,self.b) = self.get_pos
        
        
    def update(self):
        (angle_hip_lateral, angle_hip, angle_knee) = self.set_pos(self.h,self.d,self.b)
        self.hip.goal_position = angle_hip
        self.knee.goal_position = angle_knee
        self.hip_lateral.goal_position = angle_hip_lateral
        
    
    def move(self,speed,cycle,*args):
        if cycle == 'go':
            if len(args)>0 and self.d+speed > args[0] : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.d = args[0]
                return True
            else :
                self.d += speed
                return False
            
                     
        if cycle == 'back':
            if self.h-speed > args[0] and self.d-speed > args[1] :
                self.d -= speed
                self.h -= speed
                return False
            
            if self.d-speed > args[1] :
                self.h = args[0]
                self.d -= speed
                return False
            
            if self.h+speed < args[2] : 
                self.d = args[1]
                self.h += speed
                return False
            else :
                self.h = args[2]
                return True
            
                    
        if cycle == 'balance':
            if len(args)>0 and mt.copysign(1,speed)*(self.b+speed) > mt.copysign(1,speed)*args[0] : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.b = args[0]
                return True
            else :
                self.b += speed
                return False
            
        if cycle=='release':
            if self.h-speed < limit : #multiplier par le signe de la vitesse pour gérer les vitesses négatives
                self.h = limit
                return True
            else :
                self.h -= speed
                return False
            
        

Attach one primitive for each leg to the robot :

In [139]:
robot.attach_primitive(Leg(robot,'leg1'), 'leg_1')
robot.attach_primitive(Leg(robot,'leg2'), 'leg_2')
robot.attach_primitive(Leg(robot,'leg3'), 'leg_3')
robot.attach_primitive(Leg(robot,'leg4'), 'leg_4')

Now start the primitives to start the control of the legs :

In [140]:
robot.leg_1.start()
robot.leg_2.start()
robot.leg_3.start()
robot.leg_4.start()

You are now able to choose for each leg (in millimeters) :
   * h : the high of the leg
   * d : the distance between horizontal of the hip and the foot (front or rear)
   * b : the balance on right or left
   * f : the knee flexion (1 or -1) 

In [32]:
# move aside the legs
robot.leg_1.b = -20
robot.leg_3.b = -20
robot.leg_2.b = 20
robot.leg_4.b = 20

In [143]:
# get down
robot.leg_1.h = 100
robot.leg_3.h = 100
robot.leg_2.h = 100
robot.leg_4.h = 100

In [144]:
# get up
robot.leg_1.h = 150
robot.leg_3.h = 150
robot.leg_2.h = 150
robot.leg_4.h = 150

whant to try to jump :

In [ ]:
for i in range(10):
    robot.leg_1.h = 100
    robot.leg_3.h = 100
    robot.leg_2.h = 100
    robot.leg_4.h = 100
    time.sleep(0.5)
    robot.leg_1.h = 150
    robot.leg_3.h = 150
    robot.leg_2.h = 150
    robot.leg_4.h = 150
    time.sleep(0.5)
    

In [117]:
robot.leg_1.h = 120
robot.leg_3.h = 120
robot.leg_2.h = 120
robot.leg_4.h = 120
#go ahead
robot.leg_1.d = -50
robot.leg_3.d = -50
robot.leg_2.d = -50
robot.leg_4.d = -50
time.sleep(0.5)
# jump
robot.leg_1.d = 50
robot.leg_3.d = 50
robot.leg_2.d = 50
robot.leg_4.d = 50

In [ ]:
robot.leg_1.stop()
robot.leg_2.stop()
robot.leg_3.stop()
robot.leg_4.stop()

In [118]:
robot.close()

In case of trouble, for example if your robot doesnt' answer to command, you can rstart the python kernel :
Menu > Kernel > Restart